# Microtask 2

## Aim

Produce a notebook showing (and producing) a list with the activity per quarter: number of new committers, submitters of issues, and submitters of pull/merge requests, number of items (commits, issues, pull/merge requests), number of repositories with new items (all of this per quarter) as a table and as a CSV file. Using Pandas for this

### Retrieving Data from OmegaUp

From the command line run Perceval on the github repositories to analyze, to produce a file with JSON documents for all its issues (the list obtained contains the pull request also), one per line (git-commits.json).


Syntax for using Perceval for Github
`perceval github owner repository [--sleep-for-rate] [-t XXXXX]`


Date of Retrieval: 1st March 2019
##### Example:

`$ perceval github --json-line -category pull_request omegaup omegaup --sleep-for-rate -t a247a6b7d506736da6d653cddc060a96bfbd9cb3 > data_source.json`

`$ perceval github --json-line -category issue omegaup omegaup --sleep-for-rate -t a247a6b7d506736da6d653cddc060a96bfbd9cb3 >> data_source.json`

Gets the information about Pull Request and Issues from Github

###### Current Problem With Perceval    
    The Existing Problem with Perceval is that when running the above command with category issue it also 
    fetches the pull request because the github api returns the pull request under issues. For Ex 
   ##### https://api.github.com/repos/omegaup/omegaup/issues/2378
    
    This is a pull request but in url you can see it is returned as issue
   
    So we have to remove the duplicates (done in __init__() function)
    
    

`$ perceval git --json-line https://github.com/omegaup/omegaup >> data_source.json`

    Gets the information about Commits from Github

----------------------------------------------------------------------------------------
--sleep-for-rate To avoid having perceval exiting when the rate limit is exceeded

-t is token for Github API

In [87]:
import json
import datetime
import re
from dateutil import parser
import pandas as pd

import warnings ## to ignore warnings that come in importing pandas
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

## Summarize Function

#### @arguments 

<b>line</b>: item to be summarized<br>
<b>type</b>: type of item(commit,issue,pull_request)

summary{
    repo,<br>
    hash(in case of commit) or uuid(in case of PR or Issue),<br>
    author,<br>
    author_date,<br>
    ....<br>
}

In [88]:
def summarize(line,type):
    repo = line['origin']
    cdata = line['data']
    if(type=='commit'):    
        summary = {
                'repo': repo,
                'hash': cdata['commit'],
                'author': cdata['Author'],
                'author_date': datetime.datetime.strptime(cdata['AuthorDate'],
                                                          "%a %b %d %H:%M:%S %Y %z"),
                'commit': cdata['Commit'],
                'created_date': datetime.datetime.strptime(cdata['CommitDate'],
                                                          "%a %b %d %H:%M:%S %Y %z"),
                'files_no': len(cdata['files']),
        }
        actions = 0
        for file in cdata['files']:
            if 'action' in file:
                actions += 1
        summary['files_action'] = actions
        if 'Merge' in cdata:
            summary['merge'] = True
        else:
            summary['merge'] = False
    elif(type=='issue'):
        summary = {
                'repo': repo,
                'uuid': line['uuid'],
                'author': cdata['user']['login'],
                'created_date': datetime.datetime.strptime(cdata['created_at'],
                                            "%Y-%m-%dT%H:%M:%SZ"),
                'closed_date':datetime.datetime.strptime(cdata['closed_at'],
                                            "%Y-%m-%dT%H:%M:%SZ") if cdata['closed_at'] else None, 
                'comments': cdata['comments'],
                'labels': cdata['labels'],
                'url': cdata['html_url'],
                'state':cdata['state']
        }
    elif(type=='pull_request'):
        summary = {
                'repo': repo,
                'uuid': line['uuid'],
                'author': cdata['user']['login'],
                'created_date': datetime.datetime.strptime(cdata['created_at'],"%Y-%m-%dT%H:%M:%SZ"),
                'closed_date': datetime.datetime.strptime(cdata['closed_at'],"%Y-%m-%dT%H:%M:%SZ")
                                            if cdata['closed_at'] else None,
                'merged_date': datetime.datetime.strptime(cdata['merged_at'],"%Y-%m-%dT%H:%M:%SZ")
                                        if cdata['merged_at'] else None,
                'comments': cdata['comments'],
                'commits': cdata['commits'],
                'additions': cdata['additions'],
                'deletions': cdata['deletions'],
                'changed_files':cdata['changed_files'],
                'url': cdata['html_url'],
                'state':cdata['state']
        }
    
    return summary

## Class Code_Changes

Takes path to the JSON file as input parameter

In [89]:
class Code_Changes:
    """"Class for Code_Changes for Git repositories.
    
    Objects are instantiated by specifying a file with the
    commits obtained by Perceval from a set of repositories.
    
    Contains individual list for Issues, Pull Requests and Commits
        
    :param path: Path to file with one Perceval JSON document per line
    """
    
    def __init__(self, path):
        
        self.changes = {'issue':[],'commit':[],'pull_request':[]}
        with open(path) as data_file:
            for data in data_file:
                line = json.loads(data)
                if(line['category'] ==  'commit'):
                    self.changes['commit'].append(summarize(line,'commit'))
                else:
                    if (line['category'] == 'pull_request'):
                        self.changes['pull_request'].append(summarize(line,'pull_request'))
                    elif ('pull_request' not in line['data']) and (line['category'] == 'issue'):
                        self.changes['issue'].append(summarize(line,'issue'))
        self.code_dataframe = {
            'commit':pd.DataFrame.from_dict(self.changes['commit']),
            'pull_request':pd.DataFrame.from_dict(self.changes['pull_request']),
            'issue':pd.DataFrame.from_dict(self.changes['issue'])
        }
    

#### Functions Available
- total_count() : returns the total number of issues till date
- count(): returns number of issues created in Period Of Time
    ###### Parameters
    - Since
    - Until
            

## Quarter Definition

In [90]:
quarters = {} #define a quarter dictonary with key a quarter#no and value is a list with since and until (contains month and date)
quarters['quarter1'] = ({'month':1,'day':1},{'month':3,'day':31})  #January 01 – March 31
quarters['quarter2'] = ({'month':4,'day':1},{'month':6,'day':30})  #April 01 – June 30
quarters['quarter3'] = ({'month':7,'day':1},{'month':9,'day':30})  #July 01 – September 30
quarters['quarter4'] = ({'month':10,'day':1},{'month':12,'day':31})  #October 01 – December 31
quarters_keys = sorted(quarters.keys(), key=lambda x:x.lower()) ## dictonary is not formed in sorted order 
                                                           ##or in the order in which it has been instantiated

## Quarter Function

### @arguments - since (year)  , until (year)  
Divides the given duration into quarters

In [91]:
def quarterize(since,until):
    quarter_duration = []
    for year in range(since,until+1):
        for key in quarters_keys:
            value = quarters[key]
            to_append = [datetime.datetime(year,value[0]['month'],value[0]['day']),datetime.datetime(year,value[1]['month'],value[1]['day'])]
#             print(to_append)
            quarter_duration.append(to_append)
    return quarter_duration

## Example of the implementation

In [92]:
code = Code_Changes('data_source.json')

In [93]:
print("Total Number Of Commits:",len(code.changes['commit']))
print("Total Number Of Pull Requests:",len(code.changes['pull_request']))
print("Total Number Of Issues:",len(code.changes['issue']))

Total Number Of Commits: 4224
Total Number Of Pull Requests: 923
Total Number Of Issues: 1484


## First Activity in The Repository

In [94]:
code.changes['commit'].sort(key = lambda x:x['created_date'].replace(tzinfo=None))  ##sorting the commits with date 
first_commit = code.changes['commit'][0]
first_commit_year = first_commit['created_date'].year

code.changes['pull_request'].sort(key = lambda x:x['created_date'].replace(tzinfo=None))  ##sorting the pull requests with date 
first_commit = code.changes['commit'][0]
first_pull_request_year = first_commit['created_date'].year

code.changes['issue'].sort(key = lambda x:x['created_date'].replace(tzinfo=None))  ##sorting the issues with date 
first_commit = code.changes['commit'][0]
first_issue_year = first_commit['created_date'].year

first_activity_year = min(first_commit_year,first_issue_year,first_pull_request_year)
current_year = 2019
print("Year of first activity: ",first_activity_year)

Year of first activity:  2010


## Activity QuarterWise

Number of Commits, PRs, Issues QuarterWise

In [95]:
quarter_durations = quarterize(2017,2018) ## get the quarter duration between the years passed as arguments

In [115]:
activity = {'issue':[],'pull_request':[],'commit':[] }
new_contributors = {'issue':[],'pull_request':[],'commit':[] }
existing_authors = {'issue':[],'pull_request':[],'commit':[] }
for quarter in quarter_durations:
    since = quarter[0]
    until = quarter[1]
    for change_type,frame in code.code_dataframe.items():
        frame['created_date'] = frame['created_date'].apply(lambda x:x.replace(tzinfo=None))
        frame = frame[(since<=frame['created_date']) & (frame['created_date']<=until)]
                    ##got the items in the quarter duration 
        new_contributions_quarter = len(set(frame['author'].unique())-set(existing_authors[change_type]))
        new_contributors[change_type].append(new_contributions_quarter)
        activity[change_type].append(frame.shape[0])
        existing_authors[change_type] = existing_authors[change_type] + frame['author'].unique().tolist()   

## Quarter Number and Their Duration

In [116]:
for number,quarter in enumerate(quarter_durations):
    print("Quarter "+str(number + 1) + ": ",quarter[0].strftime('%d %B %Y'),"to",quarter[1].strftime('%d %B %Y'))

Quarter 1:  01 January 2017 to 31 March 2017
Quarter 2:  01 April 2017 to 30 June 2017
Quarter 3:  01 July 2017 to 30 September 2017
Quarter 4:  01 October 2017 to 31 December 2017
Quarter 5:  01 January 2018 to 31 March 2018
Quarter 6:  01 April 2018 to 30 June 2018
Quarter 7:  01 July 2018 to 30 September 2018
Quarter 8:  01 October 2018 to 31 December 2018


## Activity List

Shows the list that contains amount of commits, pull requests and issue per quarter in the given years

In [113]:
for key,value in activity.items():
    print(key)
    print("---------")
    for i in range(len(value)):
        print("Quarter "+str(i+1)+": "+str(value[i]))

issue
---------
Quarter 1: 75
Quarter 2: 104
Quarter 3: 63
Quarter 4: 90
Quarter 5: 111
Quarter 6: 128
Quarter 7: 36
Quarter 8: 29
commit
---------
Quarter 1: 102
Quarter 2: 93
Quarter 3: 65
Quarter 4: 107
Quarter 5: 130
Quarter 6: 115
Quarter 7: 61
Quarter 8: 58
pull_request
---------
Quarter 1: 73
Quarter 2: 76
Quarter 3: 63
Quarter 4: 98
Quarter 5: 114
Quarter 6: 120
Quarter 7: 51
Quarter 8: 63


## New Authors 

Shows the list that contains number of new contributors that has contributed in form commits, pull requests and issue per quarter in the given years

In [114]:
for key,value in new_contributors.items():
    print(key)
    print("---------")
    for i in range(len(value)):
        print("Quarter "+str(i+1)+": "+str(value[i]))

issue
---------
Quarter 1: 13
Quarter 2: 12
Quarter 3: 6
Quarter 4: 5
Quarter 5: 9
Quarter 6: 2
Quarter 7: 6
Quarter 8: 3
commit
---------
Quarter 1: 10
Quarter 2: 5
Quarter 3: 6
Quarter 4: 4
Quarter 5: 12
Quarter 6: 7
Quarter 7: 1
Quarter 8: 1
pull_request
---------
Quarter 1: 10
Quarter 2: 5
Quarter 3: 7
Quarter 4: 2
Quarter 5: 14
Quarter 6: 6
Quarter 7: 0
Quarter 8: 1


## Writing CSV
fields = ['Quarter','Since','Until','Commits','PRs','Issues','New PR Submitters','New Issue Submitters','New Commiters']

In [121]:
quarter_durations

[[datetime.datetime(2017, 1, 1, 0, 0), datetime.datetime(2017, 3, 31, 0, 0)],
 [datetime.datetime(2017, 4, 1, 0, 0), datetime.datetime(2017, 6, 30, 0, 0)],
 [datetime.datetime(2017, 7, 1, 0, 0), datetime.datetime(2017, 9, 30, 0, 0)],
 [datetime.datetime(2017, 10, 1, 0, 0), datetime.datetime(2017, 12, 31, 0, 0)],
 [datetime.datetime(2018, 1, 1, 0, 0), datetime.datetime(2018, 3, 31, 0, 0)],
 [datetime.datetime(2018, 4, 1, 0, 0), datetime.datetime(2018, 6, 30, 0, 0)],
 [datetime.datetime(2018, 7, 1, 0, 0), datetime.datetime(2018, 9, 30, 0, 0)],
 [datetime.datetime(2018, 10, 1, 0, 0), datetime.datetime(2018, 12, 31, 0, 0)]]

In [139]:
dataframe_csv = pd.DataFrame()
fields = ['Quarter','Since','Until','Commits','PRs','Issues','New PR Submitters','New Issue Submitters','New Commiters']
dataframe_csv['Quarter'] = ["Quarter " + str(x) for x in range(len(quarter_durations))]
dataframe_csv['Since'] = [quarter_durations[x][0].strftime('%d %B %Y') for x in range(len(quarter_durations))]
dataframe_csv['Until'] = [quarter_durations[x][1].strftime('%d %B %Y') for x in range(len(quarter_durations))]
dataframe_csv['Commits'] = activity['commit']
dataframe_csv['PRs'] = activity['pull_request']
dataframe_csv['Issues'] = activity['issue']
dataframe_csv['New PR Submitters'] = new_contributors['pull_request']
dataframe_csv['New Issue Submitters'] = new_contributors['issue']
dataframe_csv['New Commiters'] = new_contributors['commit']

dataframe_csv.to_csv('microtask2-quarterwise-pandas.csv',index=None)

In [140]:
pd.read_csv('microtask2-quarterwise-pandas.csv')

,Quarter,Since,Until,Commits,PRs,Issues,New PR Submitters,New Issue Submitters,New Commiters
0,Quarter 0,01 January 2017,31 March 2017,102,73,75,10,13,10
1,Quarter 1,01 April 2017,30 June 2017,93,76,104,5,12,5
2,Quarter 2,01 July 2017,30 September 2017,65,63,63,7,6,6
3,Quarter 3,01 October 2017,31 December 2017,107,98,90,2,5,4
4,Quarter 4,01 January 2018,31 March 2018,130,114,111,14,9,12
5,Quarter 5,01 April 2018,30 June 2018,115,120,128,6,2,7
6,Quarter 6,01 July 2018,30 September 2018,61,51,36,0,6,1
7,Quarter 7,01 October 2018,31 December 2018,58,63,29,1,3,1
